In [14]:
import pandas as pd
import numpy as np
import json
import ast

## Datasets & Cleaning

In [15]:
igdb_data_1 = pd.read_csv("../assets/raw/igdb_part1.csv")
igdb_data_2 = pd.read_csv("../assets/raw/igdb_part2.csv")
igdb_data_3 = pd.read_csv("../assets/raw/igdb_part3.csv")
igdb_data_merged = pd.concat([igdb_data_1, igdb_data_2, igdb_data_3], ignore_index=True)

/var/folders/lc/9_dv_jhs555875nqm1qhl9tr0000gn/T/ipykernel_80734/3802195039.py:1: DtypeWarning: Columns (53) have mixed types. Specify dtype option on import or set low_memory=False.
  igdb_data_1 = pd.read_csv("../assets/raw/igdb_part1.csv")
/var/folders/lc/9_dv_jhs555875nqm1qhl9tr0000gn/T/ipykernel_80734/3802195039.py:2: DtypeWarning: Columns (53) have mixed types. Specify dtype option on import or set low_memory=False.
  igdb_data_2 = pd.read_csv("../assets/raw/igdb_part2.csv")
/var/folders/lc/9_dv_jhs555875nqm1qhl9tr0000gn/T/ipykernel_80734/3802195039.py:3: DtypeWarning: Columns (53) have mixed types. Specify dtype option on import or set low_memory=False.
  igdb_data_3 = pd.read_csv("../assets/raw/igdb_part3.csv")


In [16]:
IGDB_fields = ['id', 'name', 'first_release_date', 'genres', 'rating', 'rating_count', 'total_rating', 'total_rating_count', 'aggregated_rating',
       'aggregated_rating_count']

igdb_data_merged_filtered = igdb_data_merged[IGDB_fields]
igdb_data_merged_filtered["first_release_date"] = pd.to_datetime(igdb_data_merged_filtered["first_release_date"], unit="s") # Formatting date

/var/folders/lc/9_dv_jhs555875nqm1qhl9tr0000gn/T/ipykernel_80734/1111918781.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  igdb_data_merged_filtered["first_release_date"] = pd.to_datetime(igdb_data_merged_filtered["first_release_date"], unit="s") # Formatting date


In [17]:
vanilla_wow = igdb_data_merged_filtered[igdb_data_merged_filtered['id']==123] # Vanilla WoW
cata_wow = igdb_data_merged_filtered[igdb_data_merged_filtered['id']==229] # Cataclysm
cata_wow

,id,name,first_release_date,genres,rating,rating_count,total_rating,total_rating_count,aggregated_rating,aggregated_rating_count
327330,229,World of Warcraft: Cataclysm,2010-12-07,[12],76.861351,198.0,83.097342,201.0,89.333333,3.0


In [18]:
with open("../data/genres.json", "r", encoding="utf-8") as f:
    genres_list = json.load(f)
    genres_list = {int(k): v for k, v in genres_list.items()}

genres_list

{31: 'Adventure',
 33: 'Arcade',
 35: 'Card & Board Game',
 4: 'Fighting',
 25: "Hack and slash/Beat 'em up",
 32: 'Indie',
 36: 'MOBA',
 7: 'Music',
 30: 'Pinball',
 8: 'Platform',
 2: 'Point-and-click',
 9: 'Puzzle',
 26: 'Quiz/Trivia',
 10: 'Racing',
 11: 'Real Time Strategy (RTS)',
 12: 'Role-playing (RPG)',
 5: 'Shooter',
 13: 'Simulator',
 14: 'Sport',
 15: 'Strategy',
 24: 'Tactical',
 16: 'Turn-based strategy (TBS)',
 34: 'Visual Novel'}

In [19]:
igdb_data_merged_filtered['genres'] = igdb_data_merged_filtered['genres'].map(lambda x: [] if pd.isna(x) else[genres_list[g] for g in x] if isinstance(x, list) else [genres_list[g] for g in ast.literal_eval(x)])
igdb_data_merged_filtered.head()

/var/folders/lc/9_dv_jhs555875nqm1qhl9tr0000gn/T/ipykernel_80734/129189592.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  igdb_data_merged_filtered['genres'] = igdb_data_merged_filtered['genres'].map(lambda x: [] if pd.isna(x) else[genres_list[g] for g in x] if isinstance(x, list) else [genres_list[g] for g in ast.literal_eval(x)])


,id,name,first_release_date,genres,rating,rating_count,total_rating,total_rating_count,aggregated_rating,aggregated_rating_count
0,165499,^_^,2012-01-14,[Adventure],NaN,NaN,NaN,NaN,NaN,NaN
1,141830,||[}}}°.•°.°•..°•°[|||{{{,2020-02-02,[Indie],NaN,NaN,NaN,NaN,NaN,NaN
2,191680,_____,2014-12-11,"[Shooter, Indie]",NaN,NaN,NaN,NaN,NaN,NaN
3,176875,__________,2018-07-05,"[Adventure, Indie]",NaN,NaN,NaN,NaN,NaN,NaN
4,92204,_-_,2018-03-14,"[Puzzle, Arcade]",90.0,0.0,90.0,0.0,NaN,NaN


In [20]:
# Filtering entries without rating
print(f"With and without rating = {len(igdb_data_merged_filtered)}")
igdb_data_merged_filtered_converted = igdb_data_merged_filtered.dropna(subset=['rating'])
print(f"With rating = {len(igdb_data_merged_filtered_converted)}")

# Filtering entries without first_release_date
print(f"With and without first_release_date = {len(igdb_data_merged_filtered_converted)}")
igdb_data_merged_filtered_converted = igdb_data_merged_filtered_converted[~igdb_data_merged_filtered_converted["first_release_date"].isna()]
print(f"With first_release_date = {len(igdb_data_merged_filtered_converted)}")

# Filtering entries before 2004
igdb_data_merged_filtered_converted = igdb_data_merged_filtered_converted[pd.to_datetime(igdb_data_merged_filtered_converted["first_release_date"]) > pd.to_datetime("2004-01-01")]
igdb_data_merged_filtered_converted.head()

With and without rating = 335086
With rating = 32671
With and without first_release_date = 32671
With first_release_date = 31702


,id,name,first_release_date,genres,rating,rating_count,total_rating,total_rating_count,aggregated_rating,aggregated_rating_count
4,92204,_-_,2018-03-14,"[Puzzle, Arcade]",90.000000,0.0,90.000000,0.0,NaN,NaN
23,1649,007 Legends,2012-10-16,[Shooter],54.172800,23.0,49.943543,30.0,45.714286,7.0
48,113188,03.04,2019-01-01,"[Adventure, Indie]",60.000000,2.0,60.000000,2.0,NaN,NaN
69,55486,0°N 0°W,2018-03-02,"[Adventure, Indie]",70.000000,4.0,60.000000,5.0,50.000000,1.0
72,9583,0rbitalis,2014-04-16,"[Puzzle, Simulator, Indie]",79.935551,5.0,77.467775,8.0,75.000000,3.0


In [21]:
igdb_data_merged_filtered_converted.sort_values(by="total_rating_count", ascending=False).head(10)

,id,name,first_release_date,genres,rating,rating_count,total_rating,total_rating_count,aggregated_rating,aggregated_rating_count
117313,1020,Grand Theft Auto V,2013-09-17,"[Shooter, Racing, Adventure]",89.554068,5200.0,88.846000,5227.0,88.137931,27.0
295137,1942,The Witcher 3: Wild Hunt,2015-05-19,"[Role-playing (RPG), Adventure]",93.969921,4754.0,92.850345,4780.0,91.730769,26.0
215199,72,Portal 2,2011-04-18,"[Platform, Puzzle, Adventure]",91.539118,3997.0,91.991781,4006.0,92.444444,9.0
283973,472,The Elder Scrolls V: Skyrim,2011-11-10,"[Role-playing (RPG), Adventure]",87.491280,3872.0,83.703973,3882.0,79.916667,10.0
215195,71,Portal,2007-10-10,"[Shooter, Platform, Puzzle]",86.445621,3609.0,83.222811,3610.0,80.000000,1.0
117301,732,Grand Theft Auto: San Andreas,2004-10-26,"[Shooter, Racing, Adventure]",90.231380,3508.0,91.687118,3514.0,93.142857,6.0
287598,1009,The Last of Us,2013-06-14,"[Shooter, Adventure]",93.097059,3063.0,92.317760,3076.0,91.538462,13.0
121518,233,Half-Life 2,2004-11-16,[Shooter],90.410008,3050.0,84.767504,3057.0,79.125000,7.0
226686,25076,Red Dead Redemption 2,2018-10-26,"[Shooter, Role-playing (RPG), Adventure]",93.276503,3016.0,93.667663,3033.0,94.058824,17.0
115004,19560,God of War,2018-04-20,"[Role-playing (RPG), Hack and slash/Beat 'em u...",92.362610,2996.0,94.296689,3022.0,96.230769,26.0


In [22]:
games_with_counter = igdb_data_merged_filtered_converted[igdb_data_merged_filtered_converted["name"].str.contains("counter", case=False, na=False)]
games_with_counter

,id,name,first_release_date,genres,rating,rating_count,total_rating,total_rating_count,aggregated_rating,aggregated_rating_count
58976,33278,CounterAttack,2016-04-21,"[Shooter, Adventure, Indie, Arcade]",80.000000,1.0,80.000000,1.0,NaN,NaN
58993,30643,Counter Fight,2016-11-14,"[Simulator, Indie]",80.000000,0.0,80.000000,0.0,NaN,NaN
58997,40553,Counter Fight: Samurai Edition,2017-06-07,"[Simulator, Indie]",70.000000,0.0,70.000000,0.0,NaN,NaN
59023,35703,Counter Spell,2015-10-05,"[Role-playing (RPG), Indie]",80.000000,0.0,80.000000,0.0,NaN,NaN
59025,7612,CounterSpy,2014-08-19,"[Shooter, Tactical, Indie]",75.046860,36.0,69.210930,44.0,63.375000,8.0
59028,242408,Counter-Strike 2,2023-09-27,"[Shooter, Tactical]",67.185101,249.0,67.185101,249.0,NaN,NaN
59031,297,Counter-Strike: Condition Zero,2004-03-23,[Shooter],74.174541,217.0,70.253937,220.0,66.333333,3.0
59032,93807,Counter-Strike: Condition Zero Deleted Scenes,2004-03-23,[Shooter],64.784238,32.0,64.784238,32.0,NaN,NaN
59033,1372,Counter-Strike: Global Offensive,2012-08-21,"[Shooter, Tactical]",81.813980,1245.0,81.406990,1250.0,81.000000,5.0
59037,16957,Counter-Strike Nexon,2014-10-07,"[Shooter, Indie]",20.907759,22.0,20.907759,22.0,NaN,NaN


In [23]:
igdb_data_merged_filtered_converted.to_csv("../assets/clean/IGDB_Clean.csv", index=False)